In [11]:
import cv2;
import os;
import shutil;
import random;
import numpy as np;
from PIL import Image;

In [12]:
from google.colab import drive;
drive.mount('/content/drive');

# path of original image(in toys_original foler)
read_path = "/content/drive/MyDrive/Deep_Learning_group_six_upload";
# the path to save dataset(in toys_in_the_room foler)
save_path = "/content/drive/MyDrive/Deep_Learning_group_six_upload";


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
def main() :
    
    brick_w = 300; # brick width
    brick_h = 300; # brick hight
    background_w = 2400; # background width
    background_h = 1200; # background hight
    toys_w = 256; # toy width
    toys_h = 256; # toy hight
    rotate_times = 4;
    
    # build background
    build_background(read_path, brick_w, brick_h, background_w, background_h, 0);
    
    # load all image file names
    image_names = load_image_names(read_path + "/toys_original", 0);
    
    # build dataset images
    rotate_angle = np.arange(0, 180, 45);
    np.savetxt(save_path + "/angle_type.txt", rotate_angle, delimiter =",", fmt ="% s");
    background = Image.open(read_path + "/background/bg.png");
    
    ans_list = build_dataset_image(read_path, save_path, background, image_names, toys_w, 
                                   toys_h, rotate_times, rotate_angle, 1);
    
    # build answer file
    build_ans_file(ans_list, save_path);
    
    return(0);


In [14]:
def build_background(path, brick_w, brick_h, background_w, background_h, Debug_mode) :
    
    # cv2
    img = cv2.imread(path + "/background/bg_download.jpg", cv2.IMREAD_UNCHANGED);
    cv2.rectangle(img, (0, 0), (img.shape[1], img.shape[0]), [255, 255, 255], 1);
    img= cv2.resize(img, (brick_w, brick_h), interpolation=cv2.INTER_AREA);
    cv2.imwrite(path + "/background/bg_frame.jpg", img);
    
    # Image
    img = Image.open(path + "/background/bg_frame.jpg");
    bg = Image.new('RGB',(background_w, background_h), '#000000');
    
    for i in range (0, int(background_w/brick_w)) :
        for j in range (0, int(background_h/brick_h)):
            w = i * brick_w;
            h = j * brick_h;
            bg.paste(img, (w, h));
    
    if (Debug_mode) :
        bg.show();
    
    bg.save(path + "/background/bg.png");
    
    return(0);

def load_image_names(path, Debug_mode) :
    
    image_names = os.listdir(path);
    
    if (Debug_mode) :
        print(image_names);
    
    return(image_names);

def build_dataset_image(read_path, save_path, background, image_names, resize_w, resize_h, 
                        rotate_times, rotate_angle, Debug_mode) :

    # delete old file in fold (toys_in_the_room)
    shutil.rmtree(save_path + "/toys_in_the_room/");
    os.mkdir(save_path + "/toys_in_the_room/");

    (bdw, bdh) = background.size;
    ans_list = [];
    
    for i in range (0, len(image_names)) : # for each image
        
        filename = image_names[i].split('.');
        random.shuffle(rotate_angle);
        if (Debug_mode) :
            print("(num{}) filename : {}".format(i, filename[0]));
            print("rotate_angle = {}".format(rotate_angle));
        
        img = Image.open(read_path + "/toys_original/" + image_names[i]);
        img = img.resize((resize_w, resize_h));
        
        for j in range (0, rotate_times) : # number of rotate angle
            
            x_bgn = random.randrange(0, bdw - resize_w);
            y_bgn = random.randrange(0, bdh - resize_h);
            if (Debug_mode) :
                print("(i = {}, j = {})\t(x, y) = ({}, {})".format(i, j, x_bgn, y_bgn));
            
            img_rotate = img.rotate(rotate_angle[j]);
            
            theta = str(rotate_angle[j]);
            image_id =  filename[0] + "_" + theta;
            
            new_img = combine_image(background, img_rotate, x_bgn, y_bgn);
            
            new_img.save(save_path + "/toys_in_the_room/" + image_id + ".png");
            ans_list.append([image_id, theta]);
    
    if(Debug_mode) :
        print("len(ans_list) = ", len(ans_list));
    
    return(ans_list);

def combine_image(background, img, w_bgn, h_bgn) :
    
    bg_copy = background.copy();
    new_img = np.array(bg_copy);
    img_array = np.array(img);
    
    for w in range (0, img_array.shape[1]) : # width
    
        for h in range (0, img_array.shape[0]) : # height
            
            if (img_array[h, w, 3]) : # alpha
                
                new_img[h_bgn + h, w_bgn + w, 0] = img_array[h, w, 0]; # B
                new_img[h_bgn + h, w_bgn + w, 1] = img_array[h, w, 1]; # G
                new_img[h_bgn + h, w_bgn + w, 2] = img_array[h, w, 2]; # R
                
    new_img = Image.fromarray(np.uint8(new_img));
    
    return(new_img);

def build_ans_file(ans_list, save_path) :
    
    header = ['ID', 'Theta'];
    ans_list.insert(0, header);
    np.savetxt(save_path + "/img_grabbing_angle.csv", ans_list, delimiter =",", fmt ="% s");
    
    return(0);


In [15]:
main();

(num0) filename : doll7
rotate_angle = [135   0  90  45]
(i = 0, j = 0)	(x, y) = (1762, 249)
(i = 0, j = 1)	(x, y) = (997, 58)
(i = 0, j = 2)	(x, y) = (2100, 595)
(i = 0, j = 3)	(x, y) = (1847, 508)
(num1) filename : logoman1
rotate_angle = [  0  90 135  45]
(i = 1, j = 0)	(x, y) = (2027, 876)
(i = 1, j = 1)	(x, y) = (1976, 202)
(i = 1, j = 2)	(x, y) = (2129, 486)
(i = 1, j = 3)	(x, y) = (1049, 683)
(num2) filename : logoman2
rotate_angle = [  0  90  45 135]
(i = 2, j = 0)	(x, y) = (1228, 825)
(i = 2, j = 1)	(x, y) = (1960, 301)
(i = 2, j = 2)	(x, y) = (647, 89)
(i = 2, j = 3)	(x, y) = (1861, 58)
(num3) filename : logoman3
rotate_angle = [ 90 135   0  45]
(i = 3, j = 0)	(x, y) = (766, 865)
(i = 3, j = 1)	(x, y) = (425, 648)
(i = 3, j = 2)	(x, y) = (1006, 294)
(i = 3, j = 3)	(x, y) = (422, 386)
(num4) filename : doll12
rotate_angle = [  0 135  90  45]
(i = 4, j = 0)	(x, y) = (45, 830)
(i = 4, j = 1)	(x, y) = (1706, 906)
(i = 4, j = 2)	(x, y) = (821, 919)
(i = 4, j = 3)	(x, y) = (879, 84